# Tema 2: Análisis sintáctico y Chunking

In [1]:
import nltk
from nltk import word_tokenize, pos_tag
from nltk.chunk import RegexpParser
from pathlib import Path

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

PATH_DATA = Path.cwd().parent / 'data'

## Ejercicio 1
Análisis sintáctico parcial (chunking) de una frase en español.

### Apartado a
Definir gramática para identificar chunks en español.

**Nota:** NLTK no soporta PoS tagging en español, por lo que comete errores (ej: "usa" etiquetado como JJ en lugar de VB).

In [2]:
frase_es = "Juan usa la bicicleta de Clara todos los días soleados."

tokens_es = word_tokenize(frase_es, language='spanish')
pos_tags_es = pos_tag(tokens_es)
print("Tokens con PoS (español):")
print(pos_tags_es)

# Gramática para identificar chunks
grammar = r"""
    NP: {<NNP><JJ><NN><NN>}    # Nombre propio + adjetivo + nombres
        {<JJ><JJ><NN>}         # Adjetivos + nombre
    PP: {<IN><NP>}             # Preposición + sintagma nominal
"""

chunk_parser = RegexpParser(grammar)
tree = chunk_parser.parse(pos_tags_es)
print("\nÁrbol de chunks:")
print(tree)

Tokens con PoS (español):
[('Juan', 'NNP'), ('usa', 'JJ'), ('la', 'NN'), ('bicicleta', 'NN'), ('de', 'IN'), ('Clara', 'NNP'), ('todos', 'CC'), ('los', 'JJ'), ('días', 'JJ'), ('soleados', 'NN'), ('.', '.')]

Árbol de chunks:
(S
  (NP Juan/NNP usa/JJ la/NN bicicleta/NN)
  de/IN
  Clara/NNP
  todos/CC
  (NP los/JJ días/JJ soleados/NN)
  ./.)


### Apartado b
Traducir la frase a inglés y aplicar la misma gramática.

In [3]:
frase_en = "Juan uses Clara's bicycle every sunny day."

tokens_en = word_tokenize(frase_en)
pos_tags_en = pos_tag(tokens_en)
print("Tokens con PoS (inglés):")
print(pos_tags_en)

tree_en = chunk_parser.parse(pos_tags_en)
print("\nÁrbol de chunks (inglés):")
print(tree_en)
print("\nNota: Se identifican menos chunks porque el PoS es más preciso en inglés.")

Tokens con PoS (inglés):
[('Juan', 'JJ'), ('uses', 'VBZ'), ('Clara', 'NNP'), ("'s", 'POS'), ('bicycle', 'NN'), ('every', 'DT'), ('sunny', 'JJ'), ('day', 'NN'), ('.', '.')]

Árbol de chunks (inglés):
(S
  Juan/JJ
  uses/VBZ
  Clara/NNP
  's/POS
  bicycle/NN
  every/DT
  sunny/JJ
  day/NN
  ./.)

Nota: Se identifican menos chunks porque el PoS es más preciso en inglés.


## Ejercicio 2
Chunking con RegexpParser para detectar diferentes sintagmas.

In [4]:
with open(PATH_DATA / 'Cycling.txt', 'r', encoding='utf-8') as f:
    text = f.read()

sentences = nltk.sent_tokenize(text)
print(f"Texto cargado con {len(sentences)} oraciones.")

Texto cargado con 6 oraciones.


### Apartado a
Chunks formados por dos nombres (NN NN).

In [5]:
grammar_all = r"""
    NN2: {<NN><NN>}              # Dos nombres
    DTNN: {<DT><NN>}             # Determinante + nombre
    PP: {<IN><DT>?<NN>}          # Preposición + (determinante opcional) + nombre
    VAN: {<VB|VBZ|VBP><JJ><NN>}  # Verbo + adjetivo + nombre
"""

chunk_parser = RegexpParser(grammar_all)

for sentence in sentences[:3]:
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    tree = chunk_parser.parse(pos_tags)
    print(f"\nOración: {sentence[:50]}...")
    print(tree)


Oración: Cycling's world governing body, the UCI, says it h...
(S
  Cycling/VBG
  's/POS
  (NN2 world/NN governing/NN)
  body/NN
  ,/,
  the/DT
  UCI/NNP
  ,/,
  says/VBZ
  it/PRP
  has/VBZ
  no/DT
  plans/NNS
  to/TO
  move/VB
  the/DT
  2025/CD
  Road/NNP
  World/NNP
  Championships/NNP
  away/RB
  from/IN
  Rwanda/NNP
  amid/IN
  the/DT
  ongoing/JJ
  conflict/NN
  in/IN
  neighbouring/VBG
  DR/NNP
  Congo/NNP
  ./.)

Oración: Rwanda is set to become the first African nation t...
(S
  Rwanda/NNP
  is/VBZ
  set/VBN
  to/TO
  become/VB
  the/DT
  first/JJ
  African/JJ
  nation/NN
  to/TO
  host/VB
  (DTNN the/DT event/NN)
  from/IN
  21-28/JJ
  September/NNP
  ./.)

Oración: The M23 rebel group has captured almost all of the...
(S
  The/DT
  M23/NNP
  (NN2 rebel/NN group/NN)
  has/VBZ
  captured/VBN
  almost/RB
  all/DT
  of/IN
  the/DT
  eastern/JJ
  Congolese/NNP
  city/NN
  of/IN
  Goma/NNP
  and/CC
  threatened/VBD
  to/TO
  continue/VB
  its/PRP$
  offensive/JJ
  to/TO
  (DTNN t

### Apartado b
Extraer y mostrar todos los chunks encontrados por tipo.

In [6]:
def extract_chunks(tree, chunk_type):
    """Extrae chunks de un tipo específico del árbol."""
    chunks = []
    for subtree in tree.subtrees():
        if subtree.label() == chunk_type:
            chunk = ' '.join([word for word, tag in subtree.leaves()])
            chunks.append(chunk)
    return chunks

all_nn2 = []
all_dtnn = []
all_pp = []
all_van = []

for sentence in sentences:
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    tree = chunk_parser.parse(pos_tags)
    
    all_nn2.extend(extract_chunks(tree, 'NN2'))
    all_dtnn.extend(extract_chunks(tree, 'DTNN'))
    all_pp.extend(extract_chunks(tree, 'PP'))
    all_van.extend(extract_chunks(tree, 'VAN'))

print("Chunks NN+NN:", all_nn2[:10])
print("\nChunks DT+NN:", all_dtnn[:10])
print("\nChunks PP:", all_pp[:10])
print("\nChunks VB+JJ+NN:", all_van[:10])

Chunks NN+NN: ['world governing', 'rebel group']

Chunks DT+NN: ['the event', 'the capital', 'the spread', 'this subject', 'no relocation', 'this time', 'a statement']

Chunks PP: ['on organisation', 'for tourism', 'of rumours']

Chunks VB+JJ+NN: []


## Ejercicio 3
Reconocimiento de Entidades Nombradas (NER) con NLTK en texto en inglés.

In [7]:
from nltk.chunk import ne_chunk

with open(PATH_DATA / 'Cycling.txt', 'r', encoding='utf-8') as f:
    text = f.read()

sentences = nltk.sent_tokenize(text)

print("Entidades nombradas encontradas:")
for sentence in sentences:
    tokens = word_tokenize(sentence)
    pos_tags = pos_tag(tokens)
    named_entities = ne_chunk(pos_tags)
    
    for subtree in named_entities.subtrees():
        if subtree.label() != 'S':
            entity = ' '.join([word for word, tag in subtree.leaves()])
            print(f"  {subtree.label()}: {entity}")

Entidades nombradas encontradas:
  ORGANIZATION: UCI
  GSP: Rwanda
  GSP: Rwanda
  GPE: African
  ORGANIZATION: M23
  GPE: Goma
  GPE: Kinshasa
  ORGANIZATION: UCI
  GPE: Kigali
  GSP: Rwanda
  ORGANIZATION: UCI
  ORGANIZATION: UCI Road
  GSP: Rwanda
  GPE: Switzerland


## Ejercicio 4
NER con NLTK en texto en español.

In [8]:
with open(PATH_DATA / 'Health_IA.txt', 'r', encoding='utf-8') as f:
    text_es = f.read()

sentences_es = nltk.sent_tokenize(text_es, language='spanish')

print("Entidades nombradas encontradas (NLTK en español):")
for sentence in sentences_es[:3]:
    tokens = word_tokenize(sentence, language='spanish')
    pos_tags = pos_tag(tokens)
    named_entities = ne_chunk(pos_tags)
    
    for subtree in named_entities.subtrees():
        if subtree.label() != 'S':
            entity = ' '.join([word for word, tag in subtree.leaves()])
            print(f"  {subtree.label()}: {entity}")

print("\nNota: El resultado NO es correcto porque NLTK no soporta NER en español.")

Entidades nombradas encontradas (NLTK en español):
  GPE: La
  ORGANIZATION: ChatGPT
  ORGANIZATION: DeepSeek
  PERSON: Está
  PERSON: Nuria Ribelles
  PERSON: Oncología Médica
  ORGANIZATION: Hospital Virgen
  GPE: Victoria
  GPE: Málaga
  GPE: El
  PERSON: Evaluación

Nota: El resultado NO es correcto porque NLTK no soporta NER en español.


## Ejercicio 5
NER con spaCy en texto en español.

In [9]:
import spacy

nlp = spacy.load('es_core_news_sm')

with open(PATH_DATA / 'Health_IA.txt', 'r', encoding='utf-8') as f:
    text_es = f.read()

doc = nlp(text_es)

print("Entidades nombradas encontradas (spaCy en español):")
for ent in doc.ents:
    print(f"  {ent.label_}: {ent.text}")

Entidades nombradas encontradas (spaCy en español):
  MISC: ChatGPT
  MISC: DeepSeek
  PER: Está
  LOC: Nuria Ribelles
  LOC: jefa de Sección de Oncología Médica del Hospital Virgen de la Victoria
  LOC: Málaga
  MISC: El uso de la inteligencia
  LOC: Sección SEOM de
  MISC: Evaluación de Resultados
  MISC: El uso de la inteligencia
  PER: mamografías
  MISC: Las herramientas


In [10]:
from collections import defaultdict

entities_by_type = defaultdict(list)

for ent in doc.ents:
    entities_by_type[ent.label_].append(ent.text)

print("Entidades agrupadas por tipo:")
for label, entities in entities_by_type.items():
    unique_entities = list(set(entities))
    print(f"\n{label} ({spacy.explain(label)}):")
    print(f"  {unique_entities[:10]}")

Entidades agrupadas por tipo:

MISC (Miscellaneous entities, e.g. events, nationalities, products or works of art):
  ['DeepSeek', 'El uso de la inteligencia', 'Las herramientas', 'Evaluación de Resultados', 'ChatGPT']

PER (Named person or family.):
  ['mamografías', 'Está']

LOC (Non-GPE locations, mountain ranges, bodies of water):
  ['Nuria Ribelles', 'Sección SEOM de', 'Málaga', 'jefa de Sección de Oncología Médica del Hospital Virgen de la Victoria']
